# Cyber Crime Science Data Analysis and Visualization Project

## Data Import

### Libraries

In [1]:
import pandas as pd
import numpy as np
import re


In [2]:
slachtofferschap_source = "data/source/slachtofferschap.csv"
age_distribution_source = "data/source/age_distribution.csv"
digitale_vaardigheden_source = "data/source/digitale_vaardigheden.csv"
education_levels_source = "data/source/education_levels.csv"
gdp_source = "data/source/gdp.csv"
bevolkingdichtheid_source = "data/source/bevolkingdichtheid.csv"
vestigingen_source = "data/source/vestigingen.csv"

In [3]:
slachtofferschap = pd.read_csv(slachtofferschap_source, sep=';')
age_distribution = pd.read_csv(age_distribution_source, sep=';')
digitale_vaardigheden = pd.read_csv(digitale_vaardigheden_source, sep=',')
education_levels = pd.read_csv(education_levels_source, sep=';')
gdp = pd.read_csv(gdp_source, sep=';')
bevolkingdichtheid = pd.read_csv(bevolkingdichtheid_source, sep=';')
vestigingen = pd.read_csv(vestigingen_source, sep=';')

## Data Cleaning

### Keep only the desired columns

In [4]:
# Calculate HighRiskShare for each province
vestigingen["HighRiskShare"] = (
    (vestigingen["J Informatie en communicatie"] + 
     vestigingen["K Financiële dienstverlening"] + 
     vestigingen["Q Gezondheids- en welzijnszorg"]) / 
    vestigingen["Totaal"]
) * 100

# Keep oply the "Regio's", "HighRiskShare" columns
vestigingen = vestigingen[["Regio's", "HighRiskShare"]]

In [5]:
bevolkingdichtheid_filtered = bevolkingdichtheid[[
    "Regio's",
    "Bevolking/Bevolkingssamenstelling op 1 januari/Bevolkingsdichtheid (aantal inwoners per km²)"
]]

bevolkingdichtheid_filtered = bevolkingdichtheid_filtered.rename(columns={
    "Bevolking/Bevolkingssamenstelling op 1 januari/Bevolkingsdichtheid (aantal inwoners per km²)": "Bevolkingsdichtheid (aantal inwoners per km²)"
})

display(bevolkingdichtheid_filtered.columns)

Index(['Regio's', 'Bevolkingsdichtheid (aantal inwoners per km²)'], dtype='object')

In [6]:
gdp_filtered = gdp[[
    "Regio's",
    "Bbp per inwoner (euro)",
]]

display(gdp_filtered.columns)

Index(['Regio's', 'Bbp per inwoner (euro)'], dtype='object')

In [7]:
education_levels_filtered = education_levels[[
    "Regio's",
    "Onderwijsniveau 5 categorieën/11 Basisonderwijs (%)",
    "Onderwijsniveau 5 categorieën/12 Vmbo, havo-, vwo-onderbouw, mbo1 (%)",
    "Onderwijsniveau 5 categorieën/21 Havo, vwo, mbo2-4 (%)",
    "Onderwijsniveau 5 categorieën/31 Hbo-, wo-bachelor (%)",
    "Onderwijsniveau 5 categorieën/32 Hbo-, wo-master, doctor (%)",
]]

education_levels_filtered = education_levels_filtered.rename(columns={
    "Regio's": "Regio's",
    "Onderwijsniveau 5 categorieën/11 Basisonderwijs (%)": "Basisonderwijs (%)",
    "Onderwijsniveau 5 categorieën/12 Vmbo, havo-, vwo-onderbouw, mbo1 (%)": "Vmbo/Havo/Vwo/Mbo1 (%)",
    "Onderwijsniveau 5 categorieën/21 Havo, vwo, mbo2-4 (%)": "Havo/Vwo/Mbo2-4 (%)",
    "Onderwijsniveau 5 categorieën/31 Hbo-, wo-bachelor (%)": "Hbo/Wo-bachelor (%)",
    "Onderwijsniveau 5 categorieën/32 Hbo-, wo-master, doctor (%)": "Hbo/Wo-master/Doctor (%)"
})



In [8]:
age_distribution = age_distribution.applymap(lambda x: x.replace('"', '') if isinstance(x, str) else x)
age_distribution.columns = age_distribution.columns.str.replace('"', '')

# Strip whitespace from column names to ensure proper matching
age_distribution.columns = age_distribution.columns.str.strip()

# Drop the 'Leeftijd' column
age_distribution_filtered = age_distribution.drop(columns=['Leeftijd'])

# First, convert all values to numeric (replace commas with dots and convert to float)
age_distribution_numeric = age_distribution_filtered.copy()
for col in age_distribution_numeric.columns:
    if col != "Regio's":
        age_distribution_numeric[col] = (
            age_distribution_numeric[col]
            .astype(str)
            .str.replace(",", ".", regex=False)
            .str.replace('"', '', regex=False)
            .str.strip()
        )
        age_distribution_numeric[col] = pd.to_numeric(age_distribution_numeric[col], errors='coerce')

display(age_distribution_filtered)

# Define the age intervals
age_bins = [
    (0, 9),
    (10, 19),
    (20, 29),
    (30, 39),
    (40, 49),
    (50, 59),
    (60, 69),
    (70, 79),
    (80, 89),
    (90, 200),  # 200 as a high upper bound for "95 jaar of ouder"
]

# Get all age columns (excluding "Regio's" and "Total" if present)
age_columns = [col for col in age_distribution_numeric.columns if col not in ["Regio's", "Totaal"]]

# Map column names to their integer age (if possible)
def extract_age(col):
    match = re.match(r"(\d+)", col)
    if match:
        return int(match.group(1))
    elif "95 jaar of ouder" in col:
        return 95
    else:
        return None

age_col_map = {col: extract_age(col) for col in age_columns}

# Prepare new columns for intervals
interval_cols = []
for start, end in age_bins:
    if end < 90:
        col_name = f"{start}-{end}"
    elif end == 89:
        col_name = f"{start}-{end}"
    else:
        col_name = f"{start}+"
    interval_cols.append(col_name)

# Sum the columns for each interval
age_distribution_intervals = pd.DataFrame()
age_distribution_intervals["Regio's"] = age_distribution_numeric["Regio's"]

for (start, end), col_name in zip(age_bins, interval_cols):
    cols_in_bin = [
        col for col, age in age_col_map.items()
        if age is not None and age >= start and age <= end
    ]
    age_distribution_intervals[col_name] = age_distribution_numeric[cols_in_bin].sum(axis=1)

# Bring the "Total" column from age_distribution_filtered (if it exists)
if "Totaal" in age_distribution_filtered.columns:
    # Convert to numeric if needed
    age_distribution_intervals["Totaal"] = pd.to_numeric(
        age_distribution_filtered["Totaal"]
        .astype(str)
        .str.replace(",", ".", regex=False)
        .str.replace('"', '', regex=False)
        .str.strip(),
        errors='coerce'
    )
else:
    # Fallback: sum all age columns (excluding "Regio's" and "Total")
    all_age_cols = [col for col in age_distribution_numeric.columns if col not in ["Regio's", "Total"]]
    age_distribution_intervals["Total"] = age_distribution_numeric[all_age_cols].sum(axis=1)


# Drop total column
age_distribution_intervals = age_distribution_intervals.drop(columns=['Totaal'], errors='ignore')


display(age_distribution_intervals)


C:\Users\Bulut\AppData\Local\Temp\ipykernel_3316\1440828931.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  age_distribution = age_distribution.applymap(lambda x: x.replace('"', '') if isinstance(x, str) else x)


,Regio's,Totaal,0 jaar,1 jaar,2 jaar,3 jaar,4 jaar,5 jaar,6 jaar,7 jaar,...,81 jaar,82 jaar,83 jaar,84 jaar,85 jaar,86 jaar,87 jaar,88 jaar,89 jaar,95 jaar of ouder
0,Groningen (PV),"598711,0","2400,5","4947,5","5294,5","4962,0","5116,0","4934,0","5113,0","5337,5",...,"3655,0","3173,0","2921,0","2747,5","2482,0","2161,0","1979,0","1709,0","1485,5","1099,0"
1,Fryslân (PV),"660727,5","2857,0","5851,5","6485,0","6052,5","6278,5","6154,0","6410,5","6464,0",...,"4637,0","4113,5","3831,5","3508,5","3199,0","2826,0","2533,5","2115,5","1775,0","1347,0"
2,Drenthe (PV),"503090,0","2022,0","4095,5","4656,5","4452,0","4415,5","4492,5","4736,5","4784,5",...,"3756,0","3374,0","3150,0","2834,5","2619,5","2197,5","2022,5","1742,5","1531,0","1089,5"
3,Overijssel (PV),"1186683,0","5475,5","11391,5","11863,0","11391,5","11641,0","11710,0","11852,5","12182,5",...,"7311,5","6434,0","6293,5","5740,0","5171,0","4535,5","3909,0","3318,5","2845,5","2198,5"
4,Flevoland (PV),"447763,5","2508,5","5231,5","5446,5","5242,0","5401,0","5384,5","5206,5","5381,5",...,"1673,5","1424,0","1294,0","1215,0","1080,0","870,0","820,5","711,5","600,0","435,5"
5,Gelderland (PV),"2141389,5","9721,5","19780,0","21405,0","20782,5","20585,0","20829,5","20963,5","21357,5",...,"13367,0","12218,0","11700,5","10417,5","9609,5","7937,5","7178,5","6276,0","5432,0","3918,5"
6,Utrecht (PV),"1393850,0","7017,5","14241,0","15969,0","14934,5","15111,0","15083,0","15329,5","15243,0",...,"7143,0","6544,5","6420,5","5724,5","5200,5","4517,0","3927,5","3434,0","2936,0","2336,0"
7,Noord-Holland (PV),"2964554,5","13742,5","28258,5","30437,5","28517,0","28675,0","28678,0","29067,5","29498,5",...,"16268,0","14708,5","14414,0","12985,0","11616,5","9677,5","8654,0","7498,0","6254,5","5105,5"
8,Zuid-Holland (PV),"3822683,0","18676,5","37931,0","40853,0","38990,5","39622,5","39337,0","39792,0","40740,0",...,"20362,0","18946,0","18784,0","16823,5","15262,0","12832,0","11379,5","10065,0","8513,5","7113,0"
9,Zeeland (PV),"391379,0","1683,0","3511,5","3762,5","3615,0","3644,0","3687,5","3729,0","3804,5",...,"2938,0","2607,5","2459,5","2356,5","2085,5","1805,5","1609,5","1418,5","1238,0","1125,5"


,Regio's,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+
0,Groningen (PV),48794.0,62461.0,101871.0,72403.0,64326.5,80752.0,75608.5,60857.0,26467.5,1099.0
1,Fryslân (PV),59921.0,75386.5,77784.5,73947.5,74154.0,96546.0,90060.5,73065.0,33665.0,1347.0
2,Drenthe (PV),43424.5,55665.0,51826.0,53510.0,56954.5,75876.0,73646.0,59529.0,27468.5,1089.5
3,Overijssel (PV),112680.5,139429.5,154161.0,143347.0,138478.0,167880.5,150276.0,116206.5,53830.5,2198.5
4,Flevoland (PV),50711.0,55749.0,56051.5,65529.0,57165.0,62191.0,53134.5,33523.0,11642.0,435.5
5,Gelderland (PV),198986.5,238951.5,270756.0,253769.5,243721.5,309561.0,284939.0,222523.0,99121.5,3918.5
6,Utrecht (PV),143728.5,160652.0,189994.0,195228.0,170459.0,189655.0,159265.0,119803.5,54086.0,2336.0
7,Noord-Holland (PV),276375.5,313543.0,402432.5,432928.5,367230.0,408368.5,351170.5,268223.5,120696.5,5105.5
8,Zuid-Holland (PV),377571.5,425256.0,518048.0,525728.0,471209.0,519604.0,451533.5,345126.5,156155.5,7113.0
9,Zeeland (PV),35514.5,41571.0,41549.0,43736.5,43908.5,57465.5,54337.0,46654.5,21881.5,1125.5


In [9]:
# Strip whitespace from column names to ensure proper matching
slachtofferschap.columns = slachtofferschap.columns.str.strip()

# Drop the 'Leeftijd' column
slachtofferschap_filtered = slachtofferschap.drop(columns=['Marges', 'Perioden'])

In [10]:
digitale_vaardigheden = digitale_vaardigheden.applymap(lambda x: x.replace('"', '') if isinstance(x, str) else x)
digitale_vaardigheden.columns = digitale_vaardigheden.columns.str.replace('"', '')

display(digitale_vaardigheden)



/var/folders/b_/0xr02dzn7x98hhm9_vd5w_900000gn/T/ipykernel_56074/2140010404.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  digitale_vaardigheden = digitale_vaardigheden.applymap(lambda x: x.replace('"', '') if isinstance(x, str) else x)


,Regio's,Meer dan basisvaardigheden,Basisvaardigheden,Geringe/geen digitale vaardigheden,Kan niet vastgeteld worden
0,Groningen (PV),"43,8","35,1","17,8","3,3"
1,Fryslân (PV),"44,3","33,9","14,8","7,0"
2,Drenthe (PV),"44,9","29,1","21,6","4,4"
3,Overijssel (PV),"47,9","28,4","20,9","2,8"
4,Flevoland (PV),"55,0","26,1","17,4",.
5,Gelderland (PV),"46,4","30,7","19,8","3,0"
6,Utrecht (PV),"57,6","26,7","13,7","2,0"
7,Noord-Holland (PV),"53,6","28,7","15,9","1,8"
8,Zuid-Holland (PV),"51,5","27,2","18,3","2,9"
9,Zeeland (PV),"39,9","28,3","28,7","3,1"


## Merge DataFrames

In [11]:
merged_df = pd.merge(
    slachtofferschap_filtered,
    age_distribution_intervals,
    left_on="Regio's",
    right_on="Regio's",
    how="left"  # or "left" if you want to keep all rows from slachtofferschap_filtered
)

display(merged_df)

,Regio's,Online criminaliteit/Totaal slachtoffers (%),Online oplichting en fraude/Totaal slachtoffers (%),Online oplichting en fraude/Aankoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Verkoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Fraude betalingsverkeer/Totaal slachtoffers (%),Online oplichting en fraude/Identiteitsfraude/Totaal slachtoffers (%),Online oplichting en fraude/Phishing/Totaal slachtoffers (%),Hacken/Totaal slachtoffers (%),Hacken/Apparaat/Totaal slachtoffers (%),...,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+,Totaal
0,Groningen (PV),16.3,8.6,6.5,1.4,1.0,0.6,1.0,6.3,2.7,...,62461.0,101871.0,72403.0,64326.5,80752.0,75608.5,60857.0,26467.5,1099.0,598711.0
1,Fryslân (PV),14.2,8.5,6.7,1.3,0.8,0.6,0.6,5.0,2.3,...,75386.5,77784.5,73947.5,74154.0,96546.0,90060.5,73065.0,33665.0,1347.0,660727.5
2,Drenthe (PV),14.8,8.8,6.6,1.7,1.4,0.6,1.0,5.9,2.8,...,55665.0,51826.0,53510.0,56954.5,75876.0,73646.0,59529.0,27468.5,1089.5,503090.0
3,Overijssel (PV),14.8,9.0,6.8,1.3,0.8,0.6,0.6,5.4,2.3,...,139429.5,154161.0,143347.0,138478.0,167880.5,150276.0,116206.5,53830.5,2198.5,1186683.0
4,Flevoland (PV),16.9,10.1,7.6,1.7,1.0,0.8,0.9,5.5,2.8,...,55749.0,56051.5,65529.0,57165.0,62191.0,53134.5,33523.0,11642.0,435.5,447763.5
5,Gelderland (PV),14.7,8.4,6.3,1.2,0.8,0.6,0.7,5.3,2.2,...,238951.5,270756.0,253769.5,243721.5,309561.0,284939.0,222523.0,99121.5,3918.5,2141389.5
6,Utrecht (PV),15.0,9.0,6.7,1.4,1.2,0.8,0.7,5.4,2.1,...,160652.0,189994.0,195228.0,170459.0,189655.0,159265.0,119803.5,54086.0,2336.0,1393850.0
7,Noord-Holland (PV),16.9,10.0,7.5,1.5,1.3,0.7,0.9,5.8,2.4,...,313543.0,402432.5,432928.5,367230.0,408368.5,351170.5,268223.5,120696.5,5105.5,2964554.5
8,Zuid-Holland (PV),15.8,9.5,6.9,1.4,1.1,0.8,0.9,5.5,2.3,...,425256.0,518048.0,525728.0,471209.0,519604.0,451533.5,345126.5,156155.5,7113.0,3822683.0
9,Zeeland (PV),15.8,9.8,7.4,1.8,0.9,0.7,1.0,5.5,2.4,...,41571.0,41549.0,43736.5,43908.5,57465.5,54337.0,46654.5,21881.5,1125.5,391379.0


In [12]:
merged_df = pd.merge(
    merged_df,
    education_levels_filtered,
    left_on="Regio's",
    right_on="Regio's",
    how="left"  # or "left" if you want to keep all rows from slachtofferschap_filtered
)

display(merged_df)

,Regio's,Online criminaliteit/Totaal slachtoffers (%),Online oplichting en fraude/Totaal slachtoffers (%),Online oplichting en fraude/Aankoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Verkoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Fraude betalingsverkeer/Totaal slachtoffers (%),Online oplichting en fraude/Identiteitsfraude/Totaal slachtoffers (%),Online oplichting en fraude/Phishing/Totaal slachtoffers (%),Hacken/Totaal slachtoffers (%),Hacken/Apparaat/Totaal slachtoffers (%),...,60-69,70-79,80-89,90+,Totaal,Basisonderwijs (%),Vmbo/Havo/Vwo/Mbo1 (%),Havo/Vwo/Mbo2-4 (%),Hbo/Wo-bachelor (%),Hbo/Wo-master/Doctor (%)
0,Groningen (PV),16.3,8.6,6.5,1.4,1.0,0.6,1.0,6.3,2.7,...,75608.5,60857.0,26467.5,1099.0,598711.0,7.4,16.3,45.1,20.0,11.2
1,Fryslân (PV),14.2,8.5,6.7,1.3,0.8,0.6,0.6,5.0,2.3,...,90060.5,73065.0,33665.0,1347.0,660727.5,7.8,19.1,47.0,19.7,6.4
2,Drenthe (PV),14.8,8.8,6.6,1.7,1.4,0.6,1.0,5.9,2.8,...,73646.0,59529.0,27468.5,1089.5,503090.0,8.4,18.9,46.5,18.7,7.6
3,Overijssel (PV),14.8,9.0,6.8,1.3,0.8,0.6,0.6,5.4,2.3,...,150276.0,116206.5,53830.5,2198.5,1186683.0,8.2,18.1,44.9,20.3,8.5
4,Flevoland (PV),16.9,10.1,7.6,1.7,1.0,0.8,0.9,5.5,2.8,...,53134.5,33523.0,11642.0,435.5,447763.5,8.6,18.5,45.6,19.6,7.6
5,Gelderland (PV),14.7,8.4,6.3,1.2,0.8,0.6,0.7,5.3,2.2,...,284939.0,222523.0,99121.5,3918.5,2141389.5,8.4,18.1,42.3,20.1,11.1
6,Utrecht (PV),15.0,9.0,6.7,1.4,1.2,0.8,0.7,5.4,2.1,...,159265.0,119803.5,54086.0,2336.0,1393850.0,7.6,15.0,36.1,22.2,19.0
7,Noord-Holland (PV),16.9,10.0,7.5,1.5,1.3,0.7,0.9,5.8,2.4,...,351170.5,268223.5,120696.5,5105.5,2964554.5,8.3,15.7,37.7,21.6,16.7
8,Zuid-Holland (PV),15.8,9.5,6.9,1.4,1.1,0.8,0.9,5.5,2.3,...,451533.5,345126.5,156155.5,7113.0,3822683.0,10.0,17.5,40.0,19.4,13.2
9,Zeeland (PV),15.8,9.8,7.4,1.8,0.9,0.7,1.0,5.5,2.4,...,54337.0,46654.5,21881.5,1125.5,391379.0,8.8,20.0,47.1,17.6,6.6


In [13]:
merged_df = pd.merge(
    merged_df,
    bevolkingdichtheid_filtered,
    left_on="Regio's",
    right_on="Regio's",
    how="left"  # or "left" if you want to keep all rows from slachtofferschap_filtered
)

display(merged_df)

,Regio's,Online criminaliteit/Totaal slachtoffers (%),Online oplichting en fraude/Totaal slachtoffers (%),Online oplichting en fraude/Aankoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Verkoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Fraude betalingsverkeer/Totaal slachtoffers (%),Online oplichting en fraude/Identiteitsfraude/Totaal slachtoffers (%),Online oplichting en fraude/Phishing/Totaal slachtoffers (%),Hacken/Totaal slachtoffers (%),Hacken/Apparaat/Totaal slachtoffers (%),...,70-79,80-89,90+,Totaal,Basisonderwijs (%),Vmbo/Havo/Vwo/Mbo1 (%),Havo/Vwo/Mbo2-4 (%),Hbo/Wo-bachelor (%),Hbo/Wo-master/Doctor (%),Bevolkingsdichtheid (aantal inwoners per km²)
0,Groningen (PV),16.3,8.6,6.5,1.4,1.0,0.6,1.0,6.3,2.7,...,60857.0,26467.5,1099.0,598711.0,7.4,16.3,45.1,20.0,11.2,257
1,Fryslân (PV),14.2,8.5,6.7,1.3,0.8,0.6,0.6,5.0,2.3,...,73065.0,33665.0,1347.0,660727.5,7.8,19.1,47.0,19.7,6.4,197
2,Drenthe (PV),14.8,8.8,6.6,1.7,1.4,0.6,1.0,5.9,2.8,...,59529.0,27468.5,1089.5,503090.0,8.4,18.9,46.5,18.7,7.6,191
3,Overijssel (PV),14.8,9.0,6.8,1.3,0.8,0.6,0.6,5.4,2.3,...,116206.5,53830.5,2198.5,1186683.0,8.2,18.1,44.9,20.3,8.5,357
4,Flevoland (PV),16.9,10.1,7.6,1.7,1.0,0.8,0.9,5.5,2.8,...,33523.0,11642.0,435.5,447763.5,8.6,18.5,45.6,19.6,7.6,315
5,Gelderland (PV),14.7,8.4,6.3,1.2,0.8,0.6,0.7,5.3,2.2,...,222523.0,99121.5,3918.5,2141389.5,8.4,18.1,42.3,20.1,11.1,430
6,Utrecht (PV),15.0,9.0,6.7,1.4,1.2,0.8,0.7,5.4,2.1,...,119803.5,54086.0,2336.0,1393850.0,7.6,15.0,36.1,22.2,19.0,935
7,Noord-Holland (PV),16.9,10.0,7.5,1.5,1.3,0.7,0.9,5.8,2.4,...,268223.5,120696.5,5105.5,2964554.5,8.3,15.7,37.7,21.6,16.7,1109
8,Zuid-Holland (PV),15.8,9.5,6.9,1.4,1.1,0.8,0.9,5.5,2.3,...,345126.5,156155.5,7113.0,3822683.0,10.0,17.5,40.0,19.4,13.2,1410
9,Zeeland (PV),15.8,9.8,7.4,1.8,0.9,0.7,1.0,5.5,2.4,...,46654.5,21881.5,1125.5,391379.0,8.8,20.0,47.1,17.6,6.6,220


In [14]:
vestigingen["Regio's"] = vestigingen["Regio's"].str.strip()
merged_df["Regio's"] = merged_df["Regio's"].str.strip()

merged_df = pd.merge(
    merged_df,
    vestigingen,
    left_on="Regio's",
    right_on="Regio's",
    how="left"  # or "left" if you want to keep all rows from slachtofferschap_filtered
)

display(merged_df)

,Regio's,Online criminaliteit/Totaal slachtoffers (%),Online oplichting en fraude/Totaal slachtoffers (%),Online oplichting en fraude/Aankoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Verkoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Fraude betalingsverkeer/Totaal slachtoffers (%),Online oplichting en fraude/Identiteitsfraude/Totaal slachtoffers (%),Online oplichting en fraude/Phishing/Totaal slachtoffers (%),Hacken/Totaal slachtoffers (%),Hacken/Apparaat/Totaal slachtoffers (%),...,80-89,90+,Totaal,Basisonderwijs (%),Vmbo/Havo/Vwo/Mbo1 (%),Havo/Vwo/Mbo2-4 (%),Hbo/Wo-bachelor (%),Hbo/Wo-master/Doctor (%),Bevolkingsdichtheid (aantal inwoners per km²),HighRiskShare
0,Groningen (PV),16.3,8.6,6.5,1.4,1.0,0.6,1.0,6.3,2.7,...,26467.5,1099.0,598711.0,7.4,16.3,45.1,20.0,11.2,257,18.563724
1,Fryslân (PV),14.2,8.5,6.7,1.3,0.8,0.6,0.6,5.0,2.3,...,33665.0,1347.0,660727.5,7.8,19.1,47.0,19.7,6.4,197,16.081363
2,Drenthe (PV),14.8,8.8,6.6,1.7,1.4,0.6,1.0,5.9,2.8,...,27468.5,1089.5,503090.0,8.4,18.9,46.5,18.7,7.6,191,18.672477
3,Overijssel (PV),14.8,9.0,6.8,1.3,0.8,0.6,0.6,5.4,2.3,...,53830.5,2198.5,1186683.0,8.2,18.1,44.9,20.3,8.5,357,19.470765
4,Flevoland (PV),16.9,10.1,7.6,1.7,1.0,0.8,0.9,5.5,2.8,...,11642.0,435.5,447763.5,8.6,18.5,45.6,19.6,7.6,315,20.733450
5,Gelderland (PV),14.7,8.4,6.3,1.2,0.8,0.6,0.7,5.3,2.2,...,99121.5,3918.5,2141389.5,8.4,18.1,42.3,20.1,11.1,430,20.371861
6,Utrecht (PV),15.0,9.0,6.7,1.4,1.2,0.8,0.7,5.4,2.1,...,54086.0,2336.0,1393850.0,7.6,15.0,36.1,22.2,19.0,935,22.358339
7,Noord-Holland (PV),16.9,10.0,7.5,1.5,1.3,0.7,0.9,5.8,2.4,...,120696.5,5105.5,2964554.5,8.3,15.7,37.7,21.6,16.7,1109,19.573005
8,Zuid-Holland (PV),15.8,9.5,6.9,1.4,1.1,0.8,0.9,5.5,2.3,...,156155.5,7113.0,3822683.0,10.0,17.5,40.0,19.4,13.2,1410,19.231222
9,Zeeland (PV),15.8,9.8,7.4,1.8,0.9,0.7,1.0,5.5,2.4,...,21881.5,1125.5,391379.0,8.8,20.0,47.1,17.6,6.6,220,17.124105


In [15]:
merged_df = pd.merge(
    merged_df,
    gdp_filtered,
    left_on="Regio's",
    right_on="Regio's",
    how="left"  # or "left" if you want to keep all rows from slachtofferschap_filtered
)

display(merged_df)

,Regio's,Online criminaliteit/Totaal slachtoffers (%),Online oplichting en fraude/Totaal slachtoffers (%),Online oplichting en fraude/Aankoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Verkoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Fraude betalingsverkeer/Totaal slachtoffers (%),Online oplichting en fraude/Identiteitsfraude/Totaal slachtoffers (%),Online oplichting en fraude/Phishing/Totaal slachtoffers (%),Hacken/Totaal slachtoffers (%),Hacken/Apparaat/Totaal slachtoffers (%),...,Havo/Vwo/Mbo2-4 (%),Hbo/Wo-bachelor (%),Hbo/Wo-master/Doctor (%),Bevolkingsdichtheid (aantal inwoners per km²),HighRiskShare,Bbp (marktprijzen) (mln euro),Bbp per inwoner (euro),"Bbp, volumemutaties (%)","Bruto toegevoegde waarde, volumemutaties (%)","Toegevoegde waarde (bruto, basisprijzen) (mln euro)"
0,Groningen (PV),16.3,8.6,6.5,1.4,1.0,0.6,1.0,6.3,2.7,...,45.1,20.0,11.2,257,18.563724,32566,55332,"1,7","2,1",29211
1,Fryslân (PV),14.2,8.5,6.7,1.3,0.8,0.6,0.6,5.0,2.3,...,47.0,19.7,6.4,197,16.081363,25490,39052,"3,5",4,22864
2,Drenthe (PV),14.8,8.8,6.6,1.7,1.4,0.6,1.0,5.9,2.8,...,46.5,18.7,7.6,191,18.672477,18434,37146,"2,8","3,2",16535
3,Overijssel (PV),14.8,9.0,6.8,1.3,0.8,0.6,0.6,5.4,2.3,...,44.9,20.3,8.5,357,19.470765,53172,45476,"3,8","4,2",47694
4,Flevoland (PV),16.9,10.1,7.6,1.7,1.0,0.8,0.9,5.5,2.8,...,45.6,19.6,7.6,315,20.733450,17303,40100,"4,3","4,8",15521
5,Gelderland (PV),14.7,8.4,6.3,1.2,0.8,0.6,0.7,5.3,2.2,...,42.3,20.1,11.1,430,20.371861,95416,45360,"3,8","4,3",85586
6,Utrecht (PV),15.0,9.0,6.7,1.4,1.2,0.8,0.7,5.4,2.1,...,36.1,22.2,19.0,935,22.358339,88660,64928,4,"4,5",79527
7,Noord-Holland (PV),16.9,10.0,7.5,1.5,1.3,0.7,0.9,5.8,2.4,...,37.7,21.6,16.7,1109,19.573005,203766,70285,"6,7","7,2",182774
8,Zuid-Holland (PV),15.8,9.5,6.9,1.4,1.1,0.8,0.9,5.5,2.3,...,40.0,19.4,13.2,1410,19.231222,200451,53597,"3,5",4,179802
9,Zeeland (PV),15.8,9.8,7.4,1.8,0.9,0.7,1.0,5.5,2.4,...,47.1,17.6,6.6,220,17.124105,17839,46205,"4,7","5,2",16001


In [16]:
merged_df = pd.merge(
    merged_df,
    digitale_vaardigheden,
    left_on="Regio's",
    right_on="Regio's",
    how="left"  # or "left" if you want to keep all rows from slachtofferschap_filtered
)

display(merged_df)

,Regio's,Online criminaliteit/Totaal slachtoffers (%),Online oplichting en fraude/Totaal slachtoffers (%),Online oplichting en fraude/Aankoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Verkoopfraude/Totaal slachtoffers (%),Online oplichting en fraude/Fraude betalingsverkeer/Totaal slachtoffers (%),Online oplichting en fraude/Identiteitsfraude/Totaal slachtoffers (%),Online oplichting en fraude/Phishing/Totaal slachtoffers (%),Hacken/Totaal slachtoffers (%),Hacken/Apparaat/Totaal slachtoffers (%),...,HighRiskShare,Bbp (marktprijzen) (mln euro),Bbp per inwoner (euro),"Bbp, volumemutaties (%)","Bruto toegevoegde waarde, volumemutaties (%)","Toegevoegde waarde (bruto, basisprijzen) (mln euro)",Meer dan basisvaardigheden,Basisvaardigheden,Geringe/geen digitale vaardigheden,Kan niet vastgeteld worden
0,Groningen (PV),16.3,8.6,6.5,1.4,1.0,0.6,1.0,6.3,2.7,...,18.563724,32566,55332,"1,7","2,1",29211,"43,8","35,1","17,8","3,3"
1,Fryslân (PV),14.2,8.5,6.7,1.3,0.8,0.6,0.6,5.0,2.3,...,16.081363,25490,39052,"3,5",4,22864,"44,3","33,9","14,8","7,0"
2,Drenthe (PV),14.8,8.8,6.6,1.7,1.4,0.6,1.0,5.9,2.8,...,18.672477,18434,37146,"2,8","3,2",16535,"44,9","29,1","21,6","4,4"
3,Overijssel (PV),14.8,9.0,6.8,1.3,0.8,0.6,0.6,5.4,2.3,...,19.470765,53172,45476,"3,8","4,2",47694,"47,9","28,4","20,9","2,8"
4,Flevoland (PV),16.9,10.1,7.6,1.7,1.0,0.8,0.9,5.5,2.8,...,20.733450,17303,40100,"4,3","4,8",15521,"55,0","26,1","17,4",.
5,Gelderland (PV),14.7,8.4,6.3,1.2,0.8,0.6,0.7,5.3,2.2,...,20.371861,95416,45360,"3,8","4,3",85586,"46,4","30,7","19,8","3,0"
6,Utrecht (PV),15.0,9.0,6.7,1.4,1.2,0.8,0.7,5.4,2.1,...,22.358339,88660,64928,4,"4,5",79527,"57,6","26,7","13,7","2,0"
7,Noord-Holland (PV),16.9,10.0,7.5,1.5,1.3,0.7,0.9,5.8,2.4,...,19.573005,203766,70285,"6,7","7,2",182774,"53,6","28,7","15,9","1,8"
8,Zuid-Holland (PV),15.8,9.5,6.9,1.4,1.1,0.8,0.9,5.5,2.3,...,19.231222,200451,53597,"3,5",4,179802,"51,5","27,2","18,3","2,9"
9,Zeeland (PV),15.8,9.8,7.4,1.8,0.9,0.7,1.0,5.5,2.4,...,17.124105,17839,46205,"4,7","5,2",16001,"39,9","28,3","28,7","3,1"


In [17]:
#TO DO:

#gdp merge : Done
#age distribution : Done
#formule toevoegen over die vestigingen: Done
#Digital Skills toevoegen: Done
#Voeg slachtofferschap scores voor andere crimes ook toe: Done



In [ ]:
#merged_df.to_csv("data/save/merged_data.csv", index=False)

## Analyse (To Do)

In [19]:
##Analyse
##Heatmaps: correlatie tussen verschillende variabelen met map van Nederland
##
#
## Suggested Analysis:
#
## 1. Correlation Analysis:
## Calculate the correlation matrix to understand relationships between variables.
#correlation_matrix = merged_df.corr()
#print("Correlation Matrix:")
#print(correlation_matrix)
#
## 2. Heatmap Visualization:
## Visualize the correlation matrix using a heatmap.
#import matplotlib.pyplot as plt
#
#plt.figure(figsize=(12, 8))
#sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
#plt.title("Correlation Heatmap")
#plt.show()
#
## 3. Regional Analysis:
## Group data by regions and calculate summary statistics.
#regional_summary = merged_df.groupby("Regio's").mean()
#print("Regional Summary:")
#print(regional_summary)
#
## 4. Scatter Plot Analysis:
## Visualize relationships between key variables, e.g., GDP per capita vs. online crime rates.
#plt.figure(figsize=(8, 6))
#sns.scatterplot(
#    data=merged_df,
#    x="Bbp per inwoner (euro)",
#    y="Online criminaliteit Totaal slachtoffers (%)",
#    hue="Bevolkingsdichtheid (aantal inwoners per km²)"
#)
#plt.title("GDP per Capita vs. Online Crime Rates")
#plt.xlabel("GDP per Capita (Euro)")
#plt.ylabel("Online Crime Rates (%)")
#plt.legend(title="Population Density")
#plt.show()
#
## 5. Digital Skills Impact:
## Analyze the impact of digital skills on online crime rates.
#plt.figure(figsize=(8, 6))
#sns.boxplot(
#    data=merged_df,
#    x="Digitale vaardigheden",
#    y="Online criminaliteit Totaal slachtoffers (%)"
#)
#plt.title("Impact of Digital Skills on Online Crime Rates")
#plt.xlabel("Digital Skills")
#plt.ylabel("Online Crime Rates (%)")
#plt.show()
#
## 6. Crime Type Analysis:
## Add analysis for other crime types if available in the dataset.
#if "Phishing Totaal slachtoffers (%)" in merged_df.columns:
#    plt.figure(figsize=(8, 6))
#    sns.scatterplot(
#        data=merged_df,
#        x="Bbp per inwoner (euro)",
#        y="Phishing Totaal slachtoffers (%)",
#        hue="Bevolkingsdichtheid (aantal inwoners per km²)"
#    )
#    plt.title("GDP per Capita vs. Phishing Crime Rates")
#    plt.xlabel("GDP per Capita (Euro)")
#    plt.ylabel("Phishing Crime Rates (%)")
#    plt.legend(title="Population Density")
#    plt.show()